In [1]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

import matplotlib.patches as patches
from matplotlib import gridspec
from matplotlib_venn import venn2, venn3

import os
import time
import glob

import numpy as np
from pymoc import MOC
import healpy as hp

from astropy.io.ascii import read
from astropy.io.votable import parse_single_table
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table, join
from astropy.io.misc import hdf5

from astropy.cosmology import FlatLambdaCDM
from astropy.cosmology import Planck15
from astropy.cosmology import z_at_value
from astropy.constants import iau2015 as const
from astropy import units as u

from herschelhelp_internal.utils import inMoc, coords_to_hpidx, flux_to_mag, mag_to_flux
from herschelhelp_internal.masterlist import find_last_ml_suffix, nb_ccplots

from pcigale.sed import SED
from pcigale.sed_modules import get_module


from utils_sf import *
import pickle
import multiprocessing as mp

import warnings
#We ignore warnings - this is a little dangerous but a huge number of warnings are generated by empty cells later
warnings.filterwarnings('ignore')


In [2]:
import gc
gc.collect()

22

## 1. Read in catalogue

In [3]:
DMU_DIR = '/home/mc741/Documents/git_hub/dmu_products/'
FIELD = 'Herschel-Stripe-82'

In [4]:
mysample = Table.read('./data/clean_sfg_sample_20210524.fits', memmap=True)

In [5]:
mysample[:5]

help_id,optband,nirband,ndet_total,field,ra,dec,hp_idx,ebv,redshift,zspec,f_spire_250,ferr_spire_250,flag_spire_250,f_spire_350,ferr_spire_350,flag_spire_350,f_spire_500,ferr_spire_500,flag_spire_500,cigale_mstar,cigale_mstar_err,cigale_sfr,cigale_sfr_err,cigale_dustlumin,cigale_dustlumin_err,cigale_dustlumin_ironly,cigale_dustlumin_ironly_err,cigale_chi2_opt,cigale_chi2_ir,cigale_chi2_red,cigale_chi2_ironly,stellarity,stellarity_origin,flag_cleaned,flag_merged,flag_gaia,flag_optnir_obs,flag_optnir_det,zspec_qual,zspec_association_flag,f_best_u,ferr_best_u,m_best_u,merr_best_u,flag_best_u,f_best_g,ferr_best_g,m_best_g,merr_best_g,flag_best_g,f_best_r,ferr_best_r,m_best_r,merr_best_r,flag_best_r,f_best_i,ferr_best_i,m_best_i,merr_best_i,flag_best_i,f_best_z,ferr_best_z,m_best_z,merr_best_z,flag_best_z,f_best_y,ferr_best_y,m_best_y,merr_best_y,flag_best_y,f_best_j,ferr_best_j,m_best_j,merr_best_j,flag_best_j,f_best_h,ferr_best_h,m_best_h,merr_best_h,flag_best_h,f_best_k,ferr_best_k,m_best_k,merr_best_k,flag_best_k,f_best_ks,ferr_best_ks,m_best_ks,merr_best_ks,flag_best_ks,f_irac_i1,ferr_irac_i1,m_irac_i1,merr_irac_i1,f_irac_i2,ferr_irac_i2,m_irac_i2,merr_irac_i2,f_irac_i3,ferr_irac_i3,m_irac_i3,merr_irac_i3,f_irac_i4,ferr_irac_i4,m_irac_i4,merr_irac_i4,f_mips_24,ferr_mips_24,flag_mips_24,f_pacs_green,ferr_pacs_green,flag_pacs_green,f_pacs_red,ferr_pacs_red,flag_pacs_red,flag_irac_i1,flag_irac_i2,flag_irac_i3,flag_irac_i4,sSFR
,,,,,deg,deg,,,,,uJy,uJy,,uJy,uJy,,uJy,uJy,,solMass,solMass,solMass / yr,solMass / yr,W,W,W,W,,,,,,,,,,,,,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,,uJy,uJy,mag,mag,uJy,uJy,mag,mag,uJy,uJy,mag,mag,uJy,uJy,mag,mag,uJy,uJy,,uJy,uJy,,uJy,uJy,,,,,,solMass / yr
bytes27,float64,float64,float64,bytes18,float64,float64,int64,float64,float64,float64,float64,float64,bool,float64,float64,bool,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes20,bool,bool,int64,int64,int64,int64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,float64,float64,bool,float64,float64,bool,bool,bool,bool,bool,float64
HELP_J000002.187-061445.916,1.0,2.0,3.0,Herschel-Stripe-82,0.009111123039974993,-6.24608767642777,284376088,0.034871156955701955,0.163,nan,179789.75,6067.265625,False,65744.6015625,7639.1484375,False,21428.58203125,8878.232421875,False,179877911299.4444,29394960727.309006,51.601867735606945,11.828451511676093,2.384601062400544e+38,2.8740710374968585e+37,1.8088417795174986e+38,6.145520781081247e+37,0.1395236409551362,0.006394075082650434,4.635404970156422,0.03465263511404564,0.05000000074505806,decam_stellarity,False,False,0,3,3,-99,False,nan,nan,nan,nan,False,159.1271308126664,0.5405060159880236,18.395639419555664,0.0036879125982522964,False,387.30652135321577,0.9056666586605819,17.42986297607422,0.002538854954764247,False,588.0144111586955,1.3059458964238544,16.976530075073242,0.0024113571271300316,False,818.1799184951855,2.398398076274358,16.617877960205078,0.0031827078200876713,False,978.6741617913505,5.982922944427278,16.423404693603516,0.006637424696236849,False,1479.2166748046875,29.609825134277344,15.974920272827148,0.021733436733484268,False,2076.770751953125,46.82686233520508,15.606528282165527,0.024481095373630524,False,nan,nan,nan,nan,False,2293.075439453125,47.70576477050781,15.498953819274902,0.02258795127272606,False,nan,nan,nan,nan,nan,nan,nan

## 2. SED Restframe

In [6]:
def get_center(bins):
    """
    """
    return (bins[:-1] + bins[1:]) / 2

In [7]:
def sed_restframe(path_url, path_sed, catalogue):
    """ 
    param.
    ---------------
    path_url: str
        path to the dmu folders (either local or remote). 
    path_sed: str
        path within the dmu folders.
    catalogue: astropy.table
        table containing at least the "help_id" of each source, and "redshift".
        
    output
    ---------------
    SEDs: astropy.Table 
        Table with the seds from CIGALE. Save as "object" column.
    """

#     path_url = "http://hedam.lam.fr/HELP/dataproducts/"
#     path_sed = "dmu28/dmu28_Herschel-Stripe-82/data/zphot/SEDs/fits/"

    gal_absent = []
    SEDs = catalogue.copy()
    SEDs.add_column(Column(data=np.full(len(SEDs), np.nan), dtype='object', 
                           name='seds'))
    
    for idx, gal in enumerate(SEDs['help_id']):
        if idx % 100 == 0:
            print('gal #', idx)
#             print('gal: ', gal)
    
        source_sed = "{}_best_model.fits".format(gal)
        file = path_url + path_sed + source_sed
        try:
            orig_spec = Table.read(file)        

        except FileNotFoundError:
            gal_absent.append(gal)
            print(gal,' : fail')
            continue
        
#         except Exception as e:
#     #         return gal, np.nan
#             pass  

        s = SED()
        redshift_s = SEDs[SEDs['help_id'] == gal]['redshift']
        wavelength_grid = orig_spec['wavelength']
        wavelength_grid *= 1. / (1. + redshift_s)
        luminosities = orig_spec['L_lambda_total']
        luminosities *= (1 + redshift_s)

        s.add_contribution("HELP_SED", wavelength_grid, luminosities)
        SEDs['seds'][SEDs['help_id']==gal] = s

    return SEDs #, gal_absent

In [8]:
def save_seds(catalogue, filename):
    """
    """
    with open(filename, 'wb') as f:
        pickle.dump({'help_id':catalogue['help_id'], 'seds': catalogue['seds']}, f)

In [9]:
def load_sed(filename):
    """
    """
    with open(filename, "rb") as f:
        file = pickle.load(f)
        cat = Table()
        cat['help_id'] = file['help_id']
        cat['seds'] = file['seds']
    
    return cat

### 2.1. Put SEDs at restframe

In [10]:
Vmax = mysample[['help_id', 'ra', 'dec', 'redshift', 'zspec']]

In [11]:
Vmax[:5]

help_id,ra,dec,redshift,zspec
,deg,deg,,
bytes27,float64,float64,float64,float64
HELP_J000002.187-061445.916,0.009111123039974993,-6.24608767642777,0.163,nan
HELP_J000004.805-063531.209,0.0200206346240566,-6.592002602896851,0.2717,nan
HELP_J002728.237-030737.134,6.867655196084523,-3.126981561972325,0.1179,0.1474596
HELP_J002728.472-011933.054,6.868631582140365,-1.3258483183865915,0.2177,nan
HELP_J002728.573-005425.046,6.869054443130524,-0.9069572023383792,0.2417,nan


In [12]:
path_url = "http://hedam.lam.fr/HELP/dataproducts/"
path_local = '/home/mc741/Documents/git_hub/dmu_products/'
path_sed = "dmu28/dmu28_Herschel-Stripe-82/data/zphot/SEDs/fits/"


In [16]:
f = open("./data/urls.txt", 'w')
for gal in Vmax['help_id']:
    source_sed = "{}_best_model.fits".format(gal)
    f.write(path_url + path_sed + source_sed + '\n')
f.close()

In [17]:
seds_pkl = sed_restframe(path_local, path_sed, Vmax)

In [16]:
seds_pkl[:5]

help_id,ra,dec,redshift,zspec,seds
,deg,deg,,,
bytes27,float64,float64,float64,float64,object
HELP_J000002.187-061445.916,0.009111123039974993,-6.24608767642777,0.163,nan,<pcigale.sed.SED object at 0x7f861c783160>
HELP_J000004.805-063531.209,0.0200206346240566,-6.592002602896851,0.2717,nan,<pcigale.sed.SED object at 0x7f861e475e50>
HELP_J002728.237-030737.134,6.867655196084523,-3.126981561972325,0.1179,0.1474596,<pcigale.sed.SED object at 0x7f861e475e80>
HELP_J002728.472-011933.054,6.868631582140365,-1.3258483183865915,0.2177,nan,<pcigale.sed.SED object at 0x7f861c7c9dc0>
HELP_J002728.573-005425.046,6.869054443130524,-0.9069572023383792,0.2417,nan,<pcigale.sed.SED object at 0x7f861c7c9fa0>


In [18]:
type(seds_pkl['seds'][0])

pcigale.sed.SED

In [55]:
save_seds(seds_pkl, './data/seds_90000.pkl')

### Recover failed seds

In [78]:
seds_ind = []
for idx, sed in enumerate(seds_pkl['seds']):
    if type(sed) == type(seds_pkl['seds'][0]):
        pass
    else:
        seds_ind.append(idx)

In [80]:
seds_pkl[seds_ind]

help_id,seds
bytes27,object
HELP_J004257.175-002053.239,nan
HELP_J004311.904+021426.727,nan
HELP_J004313.454+002051.782,nan
HELP_J004349.193-001135.790,nan
HELP_J004354.915+034412.572,nan
HELP_J004406.412+021413.023,nan
HELP_J004418.480+024343.065,nan
HELP_J004419.009+023815.232,nan
HELP_J004426.404+002628.569,nan


In [83]:
f = open("./data/urls_failed.txt", 'w')
for gal in seds_pkl['help_id'][seds_ind]:
    source_sed = "{}_best_model.fits".format(gal)
    f.write(path_url + path_sed + source_sed + '\n')
f.close()

In [90]:
SEDs_ = sed_restframe(path_local, path_sed, Vmax[seds_ind])

gal # 0
gal # 100
gal # 200
gal # 300
gal # 400


In [91]:
for idx, gal in enumerate(SEDs_['help_id']):
    try:
        seds_pkl['seds'][seds_pkl['help_id'] == gal] = SEDs_['seds'][idx]
    except:
        pass
    

In [92]:
seds_ind2 = []
for idx, sed in enumerate(seds_pkl['seds']):
    if type(sed) == type(seds_pkl['seds'][0]):
        pass
    else:
        seds_ind2.append(idx)

In [93]:
seds_pkl[seds_ind2]

help_id,seds
bytes27,object


In [94]:
save_seds(seds_pkl, './data/seds_restframe_20210607.pkl')